In [1]:

%pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
%pip install -q opencv-python-headless tqdm

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import kagglehub

import torch
from torch.utils.data import Dataset, DataLoader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [6]:
path = kagglehub.dataset_download("soumicksarker/ipn-hand-dataset")
print(path)
mask_path = kagglehub.dataset_download("johnsonyang123321/ipn-hand-dataset-segmentation-masks")
print(mask_path)
df = pd.read_csv(os.path.join(path, "metadata.csv"))
df.head()

C:\Users\zxjia\.cache\kagglehub\datasets\soumicksarker\ipn-hand-dataset\versions\7
C:\Users\zxjia\.cache\kagglehub\datasets\johnsonyang123321\ipn-hand-dataset-segmentation-masks\versions\1


,Video Name,Frames,Sex,Hand,Background,Illumination,People in Scene,Background Motion,Set,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,1CM1_4_R__229,3751,W,Right,Clutter,Stable,Single,Static,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1CM1_4_R__230,3684,W,Right,Clutter,Stable,Single,Static,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1CM1_4_R__231,3747,W,Right,Clutter,Stable,Single,Static,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1CM1_4_R__232,3858,W,Right,Clutter,Stable,Single,Static,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1CM42_11_R__205,3686,M,Right,Plain,Stable,Single,Static,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(df.columns)


Index(['Video Name', 'Frames', 'Sex', 'Hand', 'Background', 'Illumination',
       'People in Scene', 'Background Motion', 'Set', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
      dtype='object')


In [4]:
import os
import cv2
import matplotlib.pyplot as plt

# Paths
video_dir = os.path.join(path, "videos/videos")
mask_dir = mask_path  # KaggleHub path to your uploaded masks

# List videos
videos = os.listdir(video_dir)
print("Videos:", videos[:5])

# Pick one video for demonstration
video_file = videos[1]
video_path = os.path.join(video_dir, video_file)
print("Testing video:", video_path)

# Open video
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_indices = [total_frames//4, total_frames//2, 3*total_frames//4]

frames = []
masks = []

for idx in frame_indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ret, frame = cap.read()
    if not ret:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame_rgb)

    # Assume mask filename matches video filename + frame index, e.g., video1_frame_001_mask.png
    mask_filename = f"{video_file}_frame_{idx:04d}_mask.png"
    mask_path_file = os.path.join(mask_dir, mask_filename)
    
    if os.path.exists(mask_path_file):
        mask_img = cv2.imread(mask_path_file, cv2.IMREAD_GRAYSCALE)
        masks.append(mask_img)
    else:
        print(f"Mask not found for frame {idx}, skipping")
        masks.append(None)

cap.release()


Videos: ['1CM1_1_R__217.avi', '1CM1_1_R__218.avi', '1CM1_1_R__219.avi', '1CM1_1_R__220.avi', '1CM1_2_R__221.avi']
Testing video: C:\Users\zxjia\.cache\kagglehub\datasets\soumicksarker\ipn-hand-dataset\versions\7\videos/videos\1CM1_1_R__218.avi
Mask not found for frame 946, skipping
Mask not found for frame 1892, skipping
Mask not found for frame 2838, skipping
